In [0]:
%run ../Transformation_Samples/Sample_Data_Generator

In [0]:
# Practice Questions:
# 1. Display the first 10 rows of the DataFrame.
# 2. Find the count of rows in the DataFrame.
# 3. Show the schema of the DataFrame.
# 4. Select and display only specific columns from the DataFrame.
# 5. Filter rows based on a condition and display the result.
# 6. Group by a column and calculate aggregate statistics.
# 7. Create a new column based on existing columns.
# 8. Sort the DataFrame by a column in descending order.
# 9. Remove duplicate rows based on a column.
# 10. Write a SQL query to select rows where a column value meets a condition.


In [0]:
# 1. Price Change Detection (Window - Lag)
# Create a dataframe that shows only the exact times when a price changed for a specific site and product.
# Requirement: Use a Window function to look at the previous 15-minute price.
# Output Columns: site_id, product_id, timestamp, current_price, previous_price, price_delta.
# Filter: Keep only rows where price_delta is not 0.

df_pricing_feed = spark.read.parquet(
    "/Volumes/main/default/volume/FuelData/PricingFeed"
)
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag

window_spec = Window.partitionBy("site_id", "product_id").orderBy("timestamp")

df_with_prev = (
    df_pricing_feed.withColumn("previous_price", lag("price").over(window_spec))
    .withColumn("previous_timestamp", lag("timestamp").over(window_spec))
    .withColumn("price_delta", col("price") - col("previous_price"))
)

result_df = df_with_prev.filter(col("price_delta") != 0).select(
    "site_id", "product_id", "timestamp", "price", "previous_price", "price_delta"
)

# display(result_df)
display(df_with_prev)

2. Cumulative Daily Revenue (Window - Running Total)
Calculate the running total of sales revenue for each site throughout the day.

Requirement: Join df_transactions with df_pricing_feed to get the transaction value. Use a Window function with rowsBetween or rangeBetween to calculate a cumulative sum.

Constraint: The running total must reset at the start of every new day (if you generated multiple days) or be partitioned strictly by site_id and date.

3. Hourly Product Price Pivot (Transformation - Pivot)
Transform the vertical pricing feed into a wide report for analysis.

Requirement: Aggregate the data to calculate the Average Price per hour.

Transformation: Pivot the product_name so that each product becomes its own column.

Output Schema: site_id, hour_of_day, Shell Super FuelSave 95, Shell V-Power Diesel, etc.

4. Competitor Strategy Audit (Complex Join)
Determine if we are strictly following our pricing rules (e.g., "MATCH", "MINUS_1_CENT").

Requirement: Join df_pricing_feed (Internal) with df_competitor_mapping to find the linked competitor, then join again with df_pricing_feed (Competitor) on timestamp.

Calculation: Create a boolean flag is_compliant.

Example: If strategy is 'MATCH', is_compliant is True only if internal_price == competitor_price.

5. Top 3 "Rush Hour" Events (Window - Rank/DenseRank)
Identify the highest volume sales events for each site.

Requirement: Rank transactions by volume for each site_id.

Constraint: Use DENSE_RANK to handle ties (e.g., if two transactions have the exact same volume).

Filter: Return only the top 3 transactions per site.

6. Smoothing Price Noise (Window - Moving Average)
The simulation adds random noise to prices. Analysts want a "smoothed" trend line.

Requirement: Calculate a Rolling Average price over the last 4 intervals (Current time + Previous 3 intervals = 1 hour window).

Window Spec: rowsBetween(-3, 0).

7. Revenue Contribution Analysis (Window - Percent of Total)
Identify "Whale" transactions.

Requirement: Calculate the total daily revenue for a site and attach it to every transaction row.

Calculation: Create a column revenue_share_percentage = (transaction_amount / total_daily_site_revenue) * 100.

Constraint: Do this without a self-join (use Window functions only).